<img src="./images/DLI_Header.png" style="width: 400px;">

# 1. 클래스 환경 개요

이 섹션에서는 클래스 환경에 대한 개요와 NVIDIA의 GPU를 모니터링하는 nvidia-smi 도구를 사용한 실험을 진행합니다.

## 목표

이번 실습의 목표는 다음과 같습니다.
* 클래스에서 활용하는 하드웨어 구성 이해
* NVIDIA GPU를 모니터링하기 위한 기본 명령어 이해
* 다른 리소스를 할당하는 간단한 테스트 스크립트 실행

# 하드웨어 개요

이 클래스는 하드웨어 리소스를 제공하고 있습니다. 사용 가능한 리소스를 살펴보겠습니다.

### CPU

In [1]:
! lscpu

Architecture:                    x86_64
CPU op-mode(s):                  32-bit, 64-bit
Byte Order:                      Little Endian
Address sizes:                   46 bits physical, 48 bits virtual
CPU(s):                          32
On-line CPU(s) list:             0-31
Thread(s) per core:              2
Core(s) per socket:              16
Socket(s):                       1
NUMA node(s):                    1
Vendor ID:                       GenuineIntel
CPU family:                      6
Model:                           79
Model name:                      Intel(R) Xeon(R) CPU E5-2686 v4 @ 2.30GHz
Stepping:                        1
CPU MHz:                         2703.664
CPU max MHz:                     3000.0000
CPU min MHz:                     1200.0000
BogoMIPS:                        4600.03
Hypervisor vendor:               Xen
Virtualization type:             full
L1d cache:                       512 KiB
L1i cache:                       512 KiB
L2 cache:                     

In [2]:
# Check the number of CPU cores
!grep 'cpu cores' /proc/cpuinfo | uniq

cpu cores	: 16


### 가능한 GPUs 확인하기

In [3]:
# Check available GPUs
! nvidia-smi

Fri Mar 28 06:59:12 2025       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:00:1B.0 Off |                    0 |
| N/A   31C    P0    38W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:00:1C.0 Off |                    0 |
| N/A   

### 인터커넥트 토폴로지 

<img style="float: right;" src="images/nvlink.png" width="200"/>

멀티 GPU 시스템 구성에서는 빠르고 확장 가능한 상호 연결이 필요한 반면에 표준 PCIe 익스프레스 대역폭은 제한되어 있습니다. 

[NVIDIA NVLink 기술](https://www.nvidia.com/en-us/data-center/nvlink/)은 멀티 GPU 시스템 구성을 위한 더 높은 대역폭, 더 많은 링크 및 향상된 확장성을 제공하는 GPU 간 직접 상호 연결을 제공합니다.

사용 가능한 인터커넥트 토폴로지를 확인하려면 `nvidia-smi topo --matrix` 를 사용합니다. 이번 교육 과정에서 우리는 GPU 기기당 4개의 링크를 받아야 합니다.

```
        GPU0    GPU1    GPU2    GPU3    CPU Affinity    NUMA Affinity
GPU0     X      NV1     NV1     NV2     0-31            N/A
GPU1    NV1      X      NV2     NV1     0-31            N/A
GPU2    NV1     NV2      X      NV2     0-31            N/A
GPU3    NV2     NV1     NV2      X      0-31            N/A

Where X= Self and NV# = Connection traversing a bonded set of # NVLinks
```

In [4]:
# Check Interconnect Topology 
! nvidia-smi topo --matrix

	GPU0	GPU1	GPU2	GPU3	CPU Affinity	NUMA Affinity
GPU0	 X 	NV1	NV1	NV2	0-31		N/A
GPU1	NV1	 X 	NV2	NV1	0-31		N/A
GPU2	NV1	NV2	 X 	NV2	0-31		N/A
GPU3	NV2	NV1	NV2	 X 	0-31		N/A

Legend:

  X    = Self
  SYS  = Connection traversing PCIe as well as the SMP interconnect between NUMA nodes (e.g., QPI/UPI)
  NODE = Connection traversing PCIe as well as the interconnect between PCIe Host Bridges within a NUMA node
  PHB  = Connection traversing PCIe as well as a PCIe Host Bridge (typically the CPU)
  PXB  = Connection traversing multiple PCIe bridges (without traversing the PCIe Host Bridge)
  PIX  = Connection traversing at most a single PCIe bridge
  NV#  = Connection traversing a bonded set of # NVLinks


`nvidia-smi nvlink --status` 명령어를 사용하여 NVLink 상태 및 기능을 확인할 수도 있습니다. 장치당 유사한 출력이 있어야 합니다.

```
GPU 0: Tesla V100-SXM2-16GB 
	 Link 0: 25.781 GB/s
	 Link 1: 25.781 GB/s
	 Link 2: 25.781 GB/s
	 Link 3: 25.781 GB/s
```

In [5]:
# Check nvlink status
! nvidia-smi nvlink --status

GPU 0: Tesla V100-SXM2-16GB (UUID: GPU-759b273c-9617-dbe0-6fc1-e185f518e0eb)
	 Link 0: 25.781 GB/s
	 Link 1: 25.781 GB/s
	 Link 2: 25.781 GB/s
	 Link 3: 25.781 GB/s
GPU 1: Tesla V100-SXM2-16GB (UUID: GPU-f1f92348-9fdd-5e49-c4f4-add72ac06f4d)
	 Link 0: 25.781 GB/s
	 Link 1: 25.781 GB/s
	 Link 2: 25.781 GB/s
	 Link 3: 25.781 GB/s
GPU 2: Tesla V100-SXM2-16GB (UUID: GPU-5489d3e6-d0d1-ec98-3237-06c066b5e680)
	 Link 0: 25.781 GB/s
	 Link 1: 25.781 GB/s
	 Link 2: 25.781 GB/s
	 Link 3: 25.781 GB/s
	 Link 4: 25.781 GB/s
GPU 3: Tesla V100-SXM2-16GB (UUID: GPU-09d5cb00-3cad-cc4c-c455-6a999ee69d28)
	 Link 0: 25.781 GB/s
	 Link 1: 25.781 GB/s
	 Link 2: 25.781 GB/s
	 Link 3: 25.781 GB/s
	 Link 4: 25.781 GB/s


### 연결성 테스트하기 

    `git clone --depth 1 --branch v11.2 https://github.com/NVIDIA/cuda-samples.git`

NVIDIA는 **p2pBandwidthLatencyTest** 애플리케이션을 제공합니다. NVLinks를 활성화/비활성화하면서 대역폭 및 지연시간을 계산하여 GPU 쌍 간에 CUDA P2P(Peer-To-Peer) 데이터 전송을 보여주는 테스트입니다. 이 도구는 CUDA 개발자에서 제공하는  [cuda-samples](https://github.com/NVIDIA/cuda-samples.git) 코드 샘플의 일부입니다 

In [6]:
!chmod 770 ./cuda-samples/bin/x86_64/linux/release/p2pBandwidthLatencyTest

In [7]:
!./cuda-samples/bin/x86_64/linux/release/p2pBandwidthLatencyTest

[P2P (Peer-to-Peer) GPU Bandwidth Latency Test]
Device: 0, Tesla V100-SXM2-16GB, pciBusID: 0, pciDeviceID: 1b, pciDomainID:0
Device: 1, Tesla V100-SXM2-16GB, pciBusID: 0, pciDeviceID: 1c, pciDomainID:0
Device: 2, Tesla V100-SXM2-16GB, pciBusID: 0, pciDeviceID: 1d, pciDomainID:0
Device: 3, Tesla V100-SXM2-16GB, pciBusID: 0, pciDeviceID: 1e, pciDomainID:0
Device=0 CAN Access Peer Device=1
Device=0 CAN Access Peer Device=2
Device=0 CAN Access Peer Device=3
Device=1 CAN Access Peer Device=0
Device=1 CAN Access Peer Device=2
Device=1 CAN Access Peer Device=3
Device=2 CAN Access Peer Device=0
Device=2 CAN Access Peer Device=1
Device=2 CAN Access Peer Device=3
Device=3 CAN Access Peer Device=0
Device=3 CAN Access Peer Device=1
Device=3 CAN Access Peer Device=2

***NOTE: In case a device doesn't have P2P access to other one, it falls back to normal memcopy procedure.
So you can see lesser Bandwidth (GB/s) and unstable Latency (us) in those cases.

P2P Connectivity Matrix
     D\D     0     1  

<h2 style="color:green;">축하합니다!</h2>

과정의 이번 섹션에 대한 정보를 검토완료했으며 다음 실습을 시작할 준비가 되었습니다.<br>


[HuggingFace로 GPT-J 6b 모델 추론하기](02_HFRunInferenceOfTheGPT-J.ipynb)로 이동합니다.